In [455]:
import numpy as np
from scipy.sparse import csr_array
from itertools import combinations, starmap

#### Objective 1
* We first want to represent p binary variables into 2^p sequence of 0 and 1
* Example: 
\[0,1,0,1\] $\rightarrow$ \[0,1,0,1,0,0,0,0,1,0,0,0,0,0\]

In [456]:
def prod(*args):
    return np.prod(args)

In [457]:
from tqdm.notebook import trange

In [458]:
# simple way
sample_array = np.array([0,1,0,1,0, 1,0,1,1,1,1,1, 0, 1, 0, 1], dtype = np.byte)
result = sample_array.copy()
n = sample_array.shape[0] + 1
for i in trange(2, n):
    m = starmap(prod, combinations(sample_array.tolist(), i))
    result = np.hstack((result, np.array(list(m), dtype = np.byte)))
simple_result = np.insert(result, 0, 1)

  0%|          | 0/15 [00:00<?, ?it/s]

In [459]:
# time the process
def simulation_sim():
    sample_array = np.array([0,1,0,1,0, 1,0,1,1,1,1,1, 0, 1, 0, 1], dtype = np.byte)
    result = sample_array.copy()
    n = sample_array.shape[0] + 1
    for i in trange(2, n):
        m = starmap(prod, combinations(sample_array.tolist(), i))
        result = np.hstack((result, np.array(list(m), dtype = np.byte)))
    simple_result = np.insert(result, 0, 1)
    return simple_result

#### Using sparse matrix

In [460]:
def sum_string(*args):
    string = args[0]
    for i in range(1, len(args)):
        string = '*'.join([string, args[i]])
    return string

In [461]:
# Hard way of expanding it using sparse matrix
sample_array = np.array([0,1,0,1,0, 1,0,1,1,1,1,1, 0, 1, 0, 1], dtype = np.byte)
result = sample_array.copy()
main_list = [f"X{i}" for i in range(len(sample_array))]
index_list = main_list.copy()
n = len(main_list) + 1
for i in trange(2, n):
    m = starmap(sum_string, combinations(main_list, i))
    index_list = index_list + list(m)

  0%|          | 0/15 [00:00<?, ?it/s]

In [463]:
len(sample_array)

16

In [464]:
ones_index_main = [f"X{i}" for i in np.where(sample_array == 1)[0]]
ones_index = ones_index_main.copy()
n = len(ones_index_main)+1
for i in trange(2, n):
    m = starmap(sum_string, combinations(ones_index_main, i))
    ones_index = ones_index + list(m)
col = np.array([index_list.index(index)+1 for index in ones_index])
col = np.insert(col, 0, 0)
row = np.zeros((len(col),), dtype = np.byte)
data = np.ones((len(col),), dtype = np.byte)
csr_result = csr_array((data, (row, col)), shape=(1, 2**16))

  0%|          | 0/9 [00:00<?, ?it/s]

## Point 1: Why we use the sparse matrix: MEMORY!

In [11]:
import sys

In [12]:
sys.getsizeof(simple_result), sys.getsizeof(csr_result)

(65640, 48)

In [20]:
simple array

SyntaxError: invalid syntax (3830031417.py, line 1)

In [24]:
def simulation_diff(sample_array, index_list = index_list):
    ones_index_main = [f"X{i}" for i in np.where(sample_array == 1)[0]]
    ones_index = ones_index_main.copy()
    n = len(ones_index_main)+1
    for i in trange(2, n):
        m = starmap(sum_string, combinations(ones_index_main, i))
        ones_index = ones_index + list(m)
    col = np.array([index_list.index(index)+1 for index in ones_index])
    col=  np.insert(col, 0, 0)
    row, data = np.zeros((len(col),), dtype = np.byte), np.ones((len(col),), dtype = np.byte)
    csr_result = csr_array((data, (row, col)), shape=(1, 2**16))
    return csr_result

## Point 2: Why we use the sparse matrix: Speed!

In [18]:
%timeit simulation_sim

21 ns ± 0.0621 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [19]:
%timeit simulation_diff

20.9 ns ± 0.0382 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# 2. SVD with sparse matrix

In [123]:
array1 = np.array([0,1,0,1,0, 1,0,1,1,1,1,1, 0, 1, 0, 1], dtype = np.byte)
array2 = np.array([0,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1], dtype = np.byte)
array3 = np.array([0,0,0,1,0,0,0,1,1,0,1,1,0,1,0,1], dtype = np.byte)
array4 = np.array([0,1,0,0,0,0,0,1,1,0,1,1,0,1,0,1], dtype = np.byte)
array5 = np.array([0,0,0,0,0,0,0,1,1,0,1,1,0,1,0,1], dtype = np.byte)
array6 = np.array([0,0,1,0,1,0,0,1,1,0,1,1,0,1,0,1], dtype = np.byte)
array7 = np.array([1,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1], dtype = np.byte)

In [124]:
csr1 = simulation_diff(array1)
csr2 = simulation_diff(array2)
csr3 = simulation_diff(array3)
csr4 = simulation_diff(array4)
csr5 = simulation_diff(array5)
csr6 = simulation_diff(array6)
csr7 = simulation_diff(array7)


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [125]:
from scipy.sparse import vstack
from scipy.sparse.linalg import svds

In [163]:
# We will have csr7 twice, because it can have min(M,N) - 1 singular vectors
csr = vstack([csr1, csr2, csr3, csr4, csr5, csr6, csr7, csr7]).astype(np.float16)

In [164]:
u3, s3, vT3 = svds(csc_matrix(csr), k = 7, which  = 'LM')

In [165]:
vT3_csc = csc_matrix(vT3)

In [166]:
from scipy.linalg import inv
inv_csc = csc_matrix(inv(vT3 @ vT3.T))

In [167]:
proj = vT3_csc.T @ inv_csc @ vT3_csc

In [168]:
csr1.shape

(1, 65536)

In [169]:
proj_1 = proj @ csr1.T

In [174]:
proj_1.rint().astype(np.byte).toarray().reshape(-1)

array([1, 0, 1, ..., 0, 0, 0], dtype=int8)

In [184]:
np.allclose(proj_1.toarray().reshape(-1), csr1.toarray().astype(np.float32), atol = 1e-3)

True

In [185]:
np.allclose(proj = vT3_csc.T @ inv_csc @ vT3_csc, csr2.toarray().astype(np.float32), atol = 1e-3)

True

# 3. Example with 4 binary variables

Suppose we have a CluserRegression() estimator that generated the following segmentation:

| segment id | barcode_id |
|:---:|:---:|
|1    | 0, 3, 2, 11, 15|
|2    | 1, 5, 8, 9, 13, 14|
|3    | 4, 6, 7, 10, 12|

In [208]:
import pandas as pd
segment_id = [1 for _ in range(5)] + [2 for _ in range(6)] + [3 for _ in range(5)]
barcode_id = [0,3,2,11,15]         + [1, 5, 8, 9, 13, 14]  + [4, 6, 7, 10, 12]
df = pd.DataFrame(zip(segment_id, barcode_id), columns = ['segment_id','barcode_id'])

In [215]:
def return_sequence(i:int, length:int)->list:
    binary = bin(i)[2:]
    while len(binary) < length:
        binary = '0' + binary
    return [int(x) for x in list(binary)]

from functools import partial

lambda_1 = partial(return_sequence, length = 4)

In [221]:
result = df.barcode_id.apply(lambda_1)

In [252]:
def simulation_sim(sample_array):
    sample_array = np.array(sample_array, dtype = np.byte)
    result = sample_array.copy()
    n = sample_array.shape[0] + 1
    for i in range(2, n):
        m = starmap(prod, combinations(sample_array.tolist(), i))
        result = np.hstack((result, np.array(list(m), dtype = np.byte)))
    simple_result = np.insert(result, 0, 1)
    return simple_result

In [254]:
result_expand = result.apply(simulation_sim)

In [429]:
def pairwise_distance(array)-> np.ndarray:
    p = len(array.reset_index(drop = True)[0])
    combinations_result = list(combinations(array, 2))
    result = []
    for t in combinations_result:
        result.append([(x-y) for x,y in zip(t[0],t[1])])
    return csc_matrix(np.array(result))

In [430]:
csc1 = pairwise_distance(result_expand[:5])
csc2 = pairwise_distance(result_expand[5:11])
csc3 = pairwise_distance(result_expand[11:])

In [431]:
csc = vstack([csc1, csc2, csc3, csc3]).astype(np.float32)

In [432]:
# parameter space is 13 (16 -3)
u3, s3, vT3 = svds(csc, k = 13, which  = 'LM')

In [433]:
A3 = u3 @ np.diag(s3) @ vT3

In [454]:
np.allclose(A3, csc.toarray(), atol = 1e-4)

True

Projection with each unit vectors to see if they are in the parameter space

In [440]:
vT3_csc = csc_matrix(vT3)
inv_csc = csc_matrix(inv(vT3 @ vT3.T))
proj = vT3_csc.T @ inv_csc @ vT3_csc

In [448]:
np.round(proj.getcol(8).toarray().reshape(-1), 3)

array([-0., -0.,  0.,  0., -0.,  0.,  0.,  0.,  1., -0.,  0., -0., -0.,
       -0.,  0.,  0.], dtype=float32)

In [428]:
sample(result_expand[1]

array([1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int8)